<a href="https://colab.research.google.com/github/PlayerDos/OneTwoThreeFour/blob/main/Load_By_Month.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from datetime import datetime as dt
import numpy as np
import calendar

In [2]:
df = pd.read_csv('/content/needDSMYears.csv', index_col=0)
df.head()

,Code,Name,ParName,Calen,_31_03_22,_01_04_22,_02_04_22,_03_04_22,_04_04_22,_05_04_22,_06_04_22,_07_04_22,_08_04_22,_09_04_22,_10_04_22,_11_04_22,_12_04_22,_13_04_22,_14_04_22,_15_04_22,_16_04_22,_17_04_22,_18_04_22,_19_04_22,_20_04_22,_21_04_22,_22_04_22,_23_04_22,_24_04_22,_25_04_22,_26_04_22,_27_04_22,_28_04_22,_29_04_22,_30_04_22,_01_05_22,_02_05_22,_03_05_22,_06_05_22,_07_05_22,...,_22_11_22,_08_12_22,_09_12_22,_10_12_22,_11_12_22,_12_12_22,_13_12_22,_14_12_22,_15_12_22,_16_12_22,_17_12_22,_18_12_22,_19_12_22,_20_12_22,_21_12_22,_22_12_22,_23_12_22,_24_12_22,_25_12_22,_26_12_22,_27_12_22,_28_12_22,_29_12_22,_30_12_22,_31_12_22,_23_11_22,_24_11_22,_25_11_22,_26_11_22,_27_11_22,_28_11_22,_29_11_22,_30_11_22,_01_12_22,_02_12_22,_03_12_22,_04_12_22,_05_12_22,_06_12_22,_07_12_22
0,0,Аксай_5этап,Аксай_5этап,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Absm,Автобетоносмеситель,Автобетоносмеситель Количество,RT_24,16,11,8,5,4,3,3,3,6,6,9,9,8,9,9,6,1,1,1,4,4,4,4,4,5,1,1,2,2,5,8,8,8,8,2,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Abtnshk,Асфальтобетонщик,Асфальтобетонщик Количество,RT_20,50,35,25,20,20,19,15,10,9,14,18,9,8,18,14,10,10,10,9,15,15,4,0,4,9,13,5,5,4,4,5,9,9,9,10,18,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,AGud,Автогудронатор,Автогудронатор Количество,RT_24,10,3,2,2,1,2,1,2,0,1,1,0,1,1,2,1,1,1,2,1,1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,AP10,Асфальтоперегружатель,Асфальтоперегружатель Количество,RT_24,6,3,2,2,2,2,2,1,1,2,2,1,1,2,2,2,2,2,1,3,3,0,0,0,1,1,1,1,0,0,1,1,1,1,2,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
# форматирование имен столбцов из str в datetime ('_01_01_22' ---> 2022-01-01 00:00:00)
for index, item in enumerate(df.columns.values):
  df.columns.values[index] = pd.to_datetime(df.columns.values[index], format='_%d_%m_%y', errors='ignore')
  # форматирование имен столбцов из datetime в date (2022-01-01 00:00:00 ---> 2022-01-01)
  if type(df.columns.values[index]) is pd._libs.tslibs.timestamps.Timestamp:
    df.columns.values[index] = dt.date(df.columns.values[index])

In [4]:
# сортировка дней
df_Days = df.select_dtypes(exclude='object')
df_Info = df.select_dtypes(include='object')
df_Days.sort_index(axis=1, inplace=True)
df = df_Info.join(df_Days)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [5]:
# разбиение массива на 2 отдельных (количество техники и трудоёмкость)
df_Amount = pd.DataFrame(columns=df.columns)
df_WorkLoadPlan = pd.DataFrame(columns=df.columns)
for index, item in enumerate(df.values[:, 2]): # ??? change '2' to 'ParName' in enumerate(df.values[:, 2])
  if 'Количество' in item:
    df_Amount.loc[df_Amount.shape[0]] =  df.values[index]
  elif 'Трудоёмкость' in item:
    df_WorkLoadPlan.loc[df_WorkLoadPlan.shape[0]] = df.values[index]
  else:
    continue

РАСЧЕТ СРЕДНИХ ЗНАЧЕНИЙ ТЕХНИКИ ПО МЕСЯЦАМ

In [6]:
# форматирование фрейма для последующих операций
df_Amount.set_index('Name', inplace=True)
df_Amount = df_Amount.T

In [7]:
# получение списка значений "год-месяц" для последующего разделения фрейма по месяцам
year_month_list = []
for i in df_Amount.index[3:]:
  year_month = str(i)[:7]
  if (year_month not in year_month_list):
    year_month_list.append(year_month)

In [8]:
# создание пустого фрейма для средних значений техники по месяцам
df_Amount_Mean = pd.DataFrame(columns=df_Amount.columns)

for i in range(len(year_month_list)):
  df = pd.DataFrame(columns=df_Amount.columns)
  for index, item in enumerate(df_Amount.index):
    if year_month_list[i] in str(item):
      df = df.append(df_Amount.iloc[index])
  year = int(year_month_list[i][:4])
  month = int(year_month_list[i][5:7])
  last_day = calendar.monthrange(year, month)[1]

  # переменные с промежуточными датами
  d_01 = dt.strptime((year_month_list[i]+'-01'), '%Y-%m-%d').date()
  d_20 = dt.strptime((year_month_list[i]+'-20'), '%Y-%m-%d').date()
  d_10 = dt.strptime((year_month_list[i]+'-'+str(last_day-20)), '%Y-%m-%d').date()
  d_30 = dt.strptime((year_month_list[i]+'-'+str(last_day)), '%Y-%m-%d').date()

  # переменные с индексами элементов, соответствующих условиям
  idx_1 = df.index[(d_01 <= df.index) & (df.index <= d_20)].copy()
  idx_2 = df.index[(d_10 < df.index) & (df.index <= d_30)].copy()

  # фреймы с рассчитанными средними за первые и последние 20 дней
  df_1 = df.loc[idx_1].mean(axis=0)
  df_2 = df.loc[idx_2].mean(axis=0)

  # слияние с выбором максимального среднего в одну строку
  row_max_mean = pd.concat([df_1, df_2], axis=1).max(axis=1)
  row_max_mean.name = calendar.month_name[month]

  # добавление строки со рассчитанными средними значениями в итоговый фрейм
  df_Amount_Mean = df_Amount_Mean.append(row_max_mean)

df_Amount_Mean.T
df_Amount_Mean

Name,Автобетоносмеситель,Асфальтобетонщик,Автогудронатор,Асфальтоперегружатель,"Автосамосвал, грузоподъемность от 22 до 27 т","Асфальтоукладчик, с шириной укладки до 3,4 м","Асфальтоукладчик, с шириной укладки от 6 до 9 м","Асфальтоукладчик, с шириной укладки от 6 до 16 м","Асфальтоукладчик, с шириной укладки от 2,5 до 6 м",Автомобиль бортовой,Автомобиль бортовой с КМУ,Автоцементовоз,Автогрейдер,Автопланировщик,Бордюроукладчик (GOMACO GT-3600; 4400),Бетонщик,Бетоноукладчик с шириной укладки до 6 м (GOMACO COMMANDER III),Бульдозер мощностью от 81 до 179 л.с,Бульдозер мощностью свыше 300 л.с.,Бульдозер мощностью от 180 до 300 л.с.,"Экскаватор гусеничный аналог мод. 180-260, Vср ковша 0,95-1,4 м3","Экскаватор гусеничный аналог мод. 270-360, Vср ковша 1,4-1,7 м3","Экскаватор гусеничный аналог мод. 370-470, Vср ковша 1,7-2,5 м3","Экскаватор колесный, Vср ковша 0,4-0,95 м3",Экскаватор-планировщик,Экскаватор-погрузчик,"Фронтальный погрузчик средний, объем ковша от 2 до 4 м3","Фронтальный погрузчик большой, объем ковша свыше 4 м3",Фреза дорожная с шириной барабана от 500 до 1400мм,Фреза дорожная с шириной барабана от 1500 мм до 2200 мм,Каток дорожный гладковальцовый,Каток дорожный комбинированный,Каток дорожный пневмоколесный,Каток грунтовый,Каток тротуарный,Автокран грузоподъемностью до 25 т,Минипогрузчик,Минитрактор,Основной рабочий,Машина для ремонта дорожных покрытий,Машина щеточная,Машина для снятия фаски,Нарезчик швов мощностью более 15 кВт,Плавильно-заливочная машина,Машина поливомоечная,Распределитель бетона,Ресайклер,Станция компрессорная,"Трактор колесный, мощностью от 51 до 100 л.с.","Текстуровщик, ширина текстурирования до 10 м",Уширитель дорожного полотна,Виброрейка,Виброплита
January,0.00,0.857143,0.00,0.00,29.071429,0.00,0.00,0.00,0.214286,1.785714,0.928571,0.75,1.285714,0.00,0.00,0.00,0.0,1.60,0.071429,3.40,0.80,9.071429,2.00,1.071429,0.00,0.75,1.428571,0.0,0.80,0.0,0.214286,0.214286,0.00,3.857143,0.00,1.285714,0.214286,0.00,35.214286,0.00,0.00,0.0,0.00,0.00,2.857143,0.0,0.40,0.00,1.35,0.0,0.00,0.0,2.10
February,0.00,0.000000,0.00,0.00,32.000000,0.00,0.00,0.00,0.000000,2.800000,1.200000,0.80,0.750000,0.00,0.00,0.00,0.0,0.70,0.000000,4.50,0.30,5.950000,2.70,1.750000,0.00,1.75,2.000000,0.0,0.40,0.0,0.000000,0.000000,0.00,1.800000,0.00,1.550000,0.000000,0.00,43.200000,0.00,0.00,0.0,0.00,0.00,2.650000,0.0,0.90,0.00,2.25,0.0,0.00,0.0,3.00
March,0.80,2.500000,0.50,0.30,24.250000,0.15,0.20,0.15,0.150000,1.900000,0.950000,0.75,0.250000,0.50,0.00,0.00,0.0,1.15,0.000000,3.95,1.20,4.050000,1.95,1.100000,1.05,1.00,1.450000,0.0,0.00,0.0,1.700000,0.850000,0.70,1.200000,0.00,0.750000,0.250000,0.00,30.250000,0.00,0.00,0.0,0.00,0.00,3.000000,0.0,1.30,0.00,1.85,0.0,0.00,0.0,2.30
April,5.50,15.150000,1.30,1.90,55.300000,0.10,0.80,1.75,0.750000,6.900000,1.400000,0.70,2.900000,0.90,0.90,7.20,0.0,5.15,0.000000,2.85,3.45,3.000000,5.25,3.300000,2.00,4.35,0.550000,0.0,0.15,0.0,13.150000,4.900000,4.50,4.450000,0.15,2.400000,2.150000,0.25,96.750000,0.00,0.00,0.0,1.30,0.00,5.250000,0.0,0.70,0.00,0.70,0.0,0.15,0.0,5.40
May,5.55,10.750000,0.95,1.40,70.200000,0.00,0.20,1.40,0.750000,3.400000,0.850000,0.35,2.650000,0.00,0.70,5.60,0.0,4.15,0.250000,3.75,3.70,4.050000,3.25,1.250000,3.40,4.65,0.000000,0.0,0.00,0.0,8.350000,3.450000,3.00,5.050000,0.00,2.000000,1.750000,0.00,81.150000,0.00,0.00,0.0,1.10,0.00,4.600000,0.0,0.35,0.00,0.35,0.0,0.05,0.0,5.25
June,1.85,4.050000,0.40,0.45,55.850000,0.00,0.15,0.45,1.100000,6.250000,1.800000,1.65,3.200000,1.10,0.15,9.45,0.4,4.85,0.500000,8.60,6.70,3.500000,10.65,3.950000,1.70,5.35,5.250000,0.0,0.65,0.0,2.700000,1.250000,1.00,7.150000,0.00,2.300000,1.200000,0.00,116.850000,0.00,0.10,0.2,2.10,0.05,8.300000,0.4,2.25,0.95,3.35,0.4,0.05,0.0,5.45
July,6.25,15.600000,0.95,1.20,96.850000,0.00,0.65,0.95,1.950000,9.250000,2.300000,0.00,5.400000,0.30,0.45,14.60,0.0,7.90,0.100000,12.35,2.40,4.600000,8.00,7.100000,0.50,10.75,2.550000,0.1,0.75,3.8,11.650000,4.900000,4.55,10.800000,2.00,4.900000,4.600000,0.30,184.800000,0.30,1.2

РАСЧЕТ ТРУДОЁМКОСТИ ПО МЕСЯЦАМ

In [9]:
# форматирование фрейма для последующих операций
df_WorkLoadPlan.set_index('Name', inplace=True)
df_WorkLoadPlan = df_WorkLoadPlan.T

In [10]:
df_T = pd.DataFrame(columns=df_WorkLoadPlan.columns)

for i in range(len(year_month_list)):
  df = pd.DataFrame(columns=df_WorkLoadPlan.columns)
  for index, item in enumerate(df_WorkLoadPlan.index):
    if year_month_list[i] in str(item):
      df = df.append(df_WorkLoadPlan.iloc[index])
  year = int(year_month_list[i][:4])
  month = int(year_month_list[i][5:7])
  last_day = calendar.monthrange(year, month)[1]

  # переменные с датами первого и последнего дня месяца
  d_01 = dt.strptime((year_month_list[i]+'-01'), '%Y-%m-%d').date()
  d_30 = dt.strptime((year_month_list[i]+'-'+str(last_day)), '%Y-%m-%d').date()

  # переменные с индексами элементов (дат), входящих в месяц
  idx = df.index[(d_01 <= df.index) & (df.index <= d_30)].copy()

  # строка с рассчитанными суммами трудоёмкости за месяц
  row_sum_workloadplan = df.loc[idx].sum(axis=0)
  row_sum_workloadplan.name = calendar.month_name[month]

  # добавление строки со рассчитанными суммами трудоёмкости в итоговый фрейм
  df_T = df_T.append(row_sum_workloadplan)

df_T = df_T.iloc[:].astype(int)
df_T

Name,Автобетоносмеситель,Асфальтобетонщик,Автогудронатор,Асфальтоперегружатель,"Автосамосвал, грузоподъемность от 22 до 27 т","Асфальтоукладчик, с шириной укладки до 3,4 м","Асфальтоукладчик, с шириной укладки от 6 до 9 м","Асфальтоукладчик, с шириной укладки от 6 до 16 м","Асфальтоукладчик, с шириной укладки от 2,5 до 6 м",Автомобиль бортовой,Автомобиль бортовой с КМУ,Автоцементовоз,Автогрейдер,Автопланировщик,Бордюроукладчик (GOMACO GT-3600; 4400),Бетонщик,Бетоноукладчик с шириной укладки до 6 м (GOMACO COMMANDER III),Бульдозер мощностью от 81 до 179 л.с,Бульдозер мощностью свыше 300 л.с.,Бульдозер мощностью от 180 до 300 л.с.,"Экскаватор гусеничный аналог мод. 180-260, Vср ковша 0,95-1,4 м3","Экскаватор гусеничный аналог мод. 270-360, Vср ковша 1,4-1,7 м3","Экскаватор гусеничный аналог мод. 370-470, Vср ковша 1,7-2,5 м3","Экскаватор колесный, Vср ковша 0,4-0,95 м3",Экскаватор-планировщик,Экскаватор-погрузчик,"Фронтальный погрузчик средний, объем ковша от 2 до 4 м3","Фронтальный погрузчик большой, объем ковша свыше 4 м3",Фреза дорожная с шириной барабана от 500 до 1400мм,Фреза дорожная с шириной барабана от 1500 мм до 2200 мм,Каток дорожный гладковальцовый,Каток дорожный комбинированный,Каток дорожный пневмоколесный,Каток грунтовый,Каток тротуарный,Автокран грузоподъемностью до 25 т,Минипогрузчик,Минитрактор,Основной рабочий,Машина для ремонта дорожных покрытий,Машина щеточная,Машина для снятия фаски,Нарезчик швов мощностью более 15 кВт,Плавильно-заливочная машина,Машина поливомоечная,Распределитель бетона,Ресайклер,Станция компрессорная,"Трактор колесный, мощностью от 51 до 100 л.с.","Текстуровщик, ширина текстурирования до 10 м",Уширитель дорожного полотна,Виброрейка,Виброплита
January,0,81,0,0,10480,0,0,0,19,245,145,123,238,0,0,0,0,650,2,1366,141,3240,648,141,0,75,584,0,285,0,19,19,0,1061,0,205,13,0,11601,0,0,0,0,0,846,0,80,0,441,0,0,0,368
February,0,0,0,0,14192,0,0,0,0,544,101,112,179,0,0,0,0,439,0,1821,53,2941,1190,307,0,346,1048,0,122,0,0,0,0,526,0,258,0,0,15675,0,0,0,0,0,864,0,209,0,717,0,0,0,646
March,133,605,34,68,11945,30,52,38,30,365,110,85,44,143,0,0,0,432,0,1724,346,1856,998,188,419,96,779,0,0,0,431,211,181,325,0,136,48,0,13068,0,0,0,0,0,850,0,249,0,655,0,0,0,424
April,1286,3949,102,442,25560,7,226,432,40,1654,104,94,874,318,184,1498,0,2316,0,1204,1516,1422,1928,579,991,914,208,0,2,0,3243,1298,1251,1747,14,435,403,81,40670,0,0,0,146,0,1607,0,270,0,270,0,14,0,1275
May,1421,2896,83,397,28929,0,26,398,78,795,68,20,819,0,91,748,0,1915,28,1681,1617,1488,1332,125,1648,1571,0,0,0,0,2240,861,804,2305,0,455,238,0,36818,0,0,0,71,0,1478,0,68,0,68,0,1,0,914
June,728,1195,26,151,25734,0,9,111,173,1718,380,236,1084,381,18,2894,112,2355,50,3399,2830,1545,4667,874,577,2245,2153,0,218,0,729,374,242,3167,0,690,281,0,53229,0,13,70,673,18,2963,112,535,178,1061,112,1,0,1300
July,1476,6264,80,265,49368,0,154,211,818,3198,576,0,2241,45,101,7353,0,3852,15,6446,672,2264,3589,1882,117,3381,1204,29,48,1882,4578,2091,1979,5758,1228,1331,1593,71,82344,84,343,251,537,237,4622,0,0,681,7,0,1228,382,2790
August,1514,7160,546,449,40485,0,334,385,556,2345,201,0,992,191,224,8707,0,1784,36,3907,630,1584,1343,547,185,2011,100,0,38,2207,5359,2313,2237,3941,968,1779,1717,0,49878,7,13,0,178,0,4207,0,0,619,0,0,968,619,2504
September,920,5545,70,298,36689,0,611,318,84,1457,264,164,546,0,104,6609,0,1716,0,2901,743,1171,985,636,0,1823,0,0,232,191,4277,1786,1732,3780,689,955,631,68,32904,0,0,0,82,0,3294,0,291,522,491,0,689,522,1382
October,482,3161,65,299,19203,0,195,299,66,983,143,204,513,511,88,736,0,1294,0,1267,582,376,791,171,0,596,0,0,601,0,2462,990,935,1940,1240,1565,331,62,28422,0,0,0,70,0,2212,0,0,0,858,0,1240,0,533


РАСЧЕТ ЗАГРУЗКИ

In [11]:
df_Load = df_T / (30 * 20 * df_Amount_Mean)
df_Load = df_Load.fillna(0)

In [12]:
# форматирование отображения числовых значений с плавающей точкой (1.000 ---> 100%)
for i in df_Load.columns:
  df_Load[i] = df_Load[i].map('{:.1%}'.format)

df_Load

Name,Автобетоносмеситель,Асфальтобетонщик,Автогудронатор,Асфальтоперегружатель,"Автосамосвал, грузоподъемность от 22 до 27 т","Асфальтоукладчик, с шириной укладки до 3,4 м","Асфальтоукладчик, с шириной укладки от 6 до 9 м","Асфальтоукладчик, с шириной укладки от 6 до 16 м","Асфальтоукладчик, с шириной укладки от 2,5 до 6 м",Автомобиль бортовой,Автомобиль бортовой с КМУ,Автоцементовоз,Автогрейдер,Автопланировщик,Бордюроукладчик (GOMACO GT-3600; 4400),Бетонщик,Бетоноукладчик с шириной укладки до 6 м (GOMACO COMMANDER III),Бульдозер мощностью от 81 до 179 л.с,Бульдозер мощностью свыше 300 л.с.,Бульдозер мощностью от 180 до 300 л.с.,"Экскаватор гусеничный аналог мод. 180-260, Vср ковша 0,95-1,4 м3","Экскаватор гусеничный аналог мод. 270-360, Vср ковша 1,4-1,7 м3","Экскаватор гусеничный аналог мод. 370-470, Vср ковша 1,7-2,5 м3","Экскаватор колесный, Vср ковша 0,4-0,95 м3",Экскаватор-планировщик,Экскаватор-погрузчик,"Фронтальный погрузчик средний, объем ковша от 2 до 4 м3","Фронтальный погрузчик большой, объем ковша свыше 4 м3",Фреза дорожная с шириной барабана от 500 до 1400мм,Фреза дорожная с шириной барабана от 1500 мм до 2200 мм,Каток дорожный гладковальцовый,Каток дорожный комбинированный,Каток дорожный пневмоколесный,Каток грунтовый,Каток тротуарный,Автокран грузоподъемностью до 25 т,Минипогрузчик,Минитрактор,Основной рабочий,Машина для ремонта дорожных покрытий,Машина щеточная,Машина для снятия фаски,Нарезчик швов мощностью более 15 кВт,Плавильно-заливочная машина,Машина поливомоечная,Распределитель бетона,Ресайклер,Станция компрессорная,"Трактор колесный, мощностью от 51 до 100 л.с.","Текстуровщик, ширина текстурирования до 10 м",Уширитель дорожного полотна,Виброрейка,Виброплита
January,0.0%,15.8%,0.0%,0.0%,60.1%,0.0%,0.0%,0.0%,14.8%,22.9%,26.0%,27.3%,30.9%,0.0%,0.0%,0.0%,0.0%,67.7%,4.7%,67.0%,29.4%,59.5%,54.0%,21.9%,0.0%,16.7%,68.1%,0.0%,59.4%,0.0%,14.8%,14.8%,0.0%,45.8%,0.0%,26.6%,10.1%,0.0%,54.9%,0.0%,0.0%,0.0%,0.0%,0.0%,49.4%,0.0%,33.3%,0.0%,54.4%,0.0%,0.0%,0.0%,29.2%
February,0.0%,0.0%,0.0%,0.0%,73.9%,0.0%,0.0%,0.0%,0.0%,32.4%,14.0%,23.3%,39.8%,0.0%,0.0%,0.0%,0.0%,104.5%,0.0%,67.4%,29.4%,82.4%,73.5%,29.2%,0.0%,33.0%,87.3%,0.0%,50.8%,0.0%,0.0%,0.0%,0.0%,48.7%,0.0%,27.7%,0.0%,0.0%,60.5%,0.0%,0.0%,0.0%,0.0%,0.0%,54.3%,0.0%,38.7%,0.0%,53.1%,0.0%,0.0%,0.0%,35.9%
March,27.7%,40.3%,11.3%,37.8%,82.1%,33.3%,43.3%,42.2%,33.3%,32.0%,19.3%,18.9%,29.3%,47.7%,0.0%,0.0%,0.0%,62.6%,0.0%,72.7%,48.1%,76.4%,85.3%,28.5%,66.5%,16.0%,89.5%,0.0%,0.0%,0.0%,42.3%,41.4%,43.1%,45.1%,0.0%,30.2%,32.0%,0.0%,72.0%,0.0%,0.0%,0.0%,0.0%,0.0%,47.2%,0.0%,31.9%,0.0%,59.0%,0.0%,0.0%,0.0%,30.7%
April,39.0%,43.4%,13.1%,38.8%,77.0%,11.7%,47.1%,41.1%,8.9%,40.0%,12.4%,22.4%,50.2%,58.9%,34.1%,34.7%,0.0%,75.0%,0.0%,70.4%,73.2%,79.0%,61.2%,29.2%,82.6%,35.0%,63.0%,0.0%,2.2%,0.0%,41.1%,44.1%,46.3%,65.4%,15.6%,30.2%,31.2%,54.0%,70.1%,0.0%,0.0%,0.0%,18.7%,0.0%,51.0%,0.0%,64.3%,0.0%,64.3%,0.0%,15.6%,0.0%,39.4%
May,42.7%,44.9%,14.6%,47.3%,68.7%,0.0%,21.7%,47.4%,17.3%,39.0%,13.3%,9.5%,51.5%,0.0%,21.7%,22.3%,0.0%,76.9%,18.7%,74.7%,72.8%,61.2%,68.3%,16.7%,80.8%,56.3%,0.0%,0.0%,0.0%,0.0%,44.7%,41.6%,44.7%,76.1%,0.0%,37.9%,22.7%,0.0%,75.6%,0.0%,0.0%,0.0%,10.8%,0.0%,53.6%,0.0%,32.4%,0.0%,32.4%,0.0%,3.3%,0.0%,29.0%
June,65.6%,49.2%,10.8%,55.9%,76.8%,0.0%,10.0%,41.1%,26.2%,45.8%,35.2%,23.8%,56.5%,57.7%,20.0%,51.0%,46.7%,80.9%,16.7%,65.9%,70.4%,73.6%,73.0%,36.9%,56.6%,69.9%,68.3%,0.0%,55.9%,0.0%,45.0%,49.9%,40.3%,73.8%,0.0%,50.0%,39.0%,0.0%,75.9%,0.0%,21.7%,58.3%,53.4%,60.0%,59.5%,46.7%,39.6%,31.2%,52.8%,46.7%,3.3%,0.0%,39.8%
July,39.4%,66.9%,14.0%,36.8%,85.0%,0.0%,39.5%,37.0%,69.9%,57.6%,41.7%,0.0%,69.2%,25.0%,37.4%,83.9%,0.0%,81.3%,25.0%,87.0%,46.7%,82.0%,74.8%,44.2%,39.0%,52.4%,78.7%,48.3%,10.7%,82.5%,65.5%,71.1%,72.5%,88.9%,102.3%,45.3%,57.7%,39.4%,74.3%,46.7%,45.7%,59.8%,57.7%,65.8%,78.6%,0.0%,0.0%,84.1%,11.7%,0.0%,102.3%,63.7%,42.7%
August,49.5%,54.7%,50.6%,35.6%,84.0%,0.0%,58.6%,32.1%,48.8%,54.3%,21.6%,0.0%,54.2%,57.9%,49.8%,83.4%,0.0%,76.2%,13.3%,77.1%,45.7%,86.6%,57.4%,

In [13]:
# добавление подназваний столбцов
df_Load.columns = pd.MultiIndex.from_product([["Загрузка"], df_Load.columns])
df_T.columns = pd.MultiIndex.from_product([["Трудоёмкость"], df_T.columns])
df_Load = df_Load.swaplevel(axis=1)
df_T = df_T.swaplevel(axis=1)

In [14]:
# слияние фреймов загрузки и трудоёмкости
df = pd.merge(df_Load, df_T, left_index=True, right_index=True)
df = df.T
df.sort_index()

January  ... December
Name                                                             ...         
Автобетоносмеситель                        Загрузка        0.0%  ...     0.0%
                                           Трудоёмкость       0  ...        0
Автогрейдер                                Загрузка       30.9%  ...    15.0%
                                           Трудоёмкость     238  ...       18
Автогудронатор                             Загрузка        0.0%  ...     0.0%
...                                                         ...  ...      ...
Экскаватор колесный, Vср ковша 0,4-0,95 м3 Трудоёмкость     141  ...      179
Экскаватор-планировщик                     Загрузка        0.0%  ...     0.0%
                                           Трудоёмкость       0  ...        0
Экскаватор-погрузчик                       Загрузка       16.7%  ...     9.3%
                                           Трудоёмкость      75  ...       42

[106 rows x 12 columns]

In [ ]:
# сохранение результатов в csv-файл
df_Load.to_csv('Load_Data.csv')

In [65]:
# сохранение результатов в xls-файл
df_Load.to_excel('Load_Data.xlsx')